In [1]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
text_docs = loader.load()

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(text_docs)
documents

[Document(metadata={'source': 'speech.txt'}, page_content='I am honored to be with you today at your commencement from one of the finest\nuniversities in the world. I never graduated from college. Truth be told, this is the closest\nI\'ve ever gotten to a college graduation. Today I want to tell you.. a story.. from my life.\nThat\'s it. No big dealâ€¦.\nMy story is about death.\nWhen I was 17, I read a quote that went something like: "If you live each day as if it was\nyour last, someday you\'ll most certainly be right." It made an impression on me, and\nsince then, for the past 33 years, I have looked in the mirror every morning and asked\nmyself: "If today were the last day of my life, would I want to do what I am about to do\ntoday?" And whenever the answer has been "No" for too many days in a row, I know I\nneed to change something.\nRemembering that I\'ll be dead soon is the most important tool I\'ve ever encountered to\nhelp me make the big choices in life. Because almost everyt

In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents,HuggingFaceBgeEmbeddings())

d:\LangChain\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000171C0ED2BF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000171C0ED1F30>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
You will be reward you positvely if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}""")

In [7]:
#Chain intro - Stuff Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000171BF3E4CD0>)

In [9]:
#retriever chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver,document_chain)

In [13]:
response = retrieval_chain.invoke({"input":"Tell me the most important concept from the speech."})
response['answer']

'The most important concept from the speech is the idea that "Remembering that you are going to die is the best way I know to avoid the trap of thinking you have something to lose. You are already naked. There is no reason not to follow your heart." This concept is emphasized throughout the speech as the key to making big choices in life and living authentically, rather than being held back by fear of failure, embarrassment, or external expectations.'